# Compute domain-integrated energy conversions for Figure 8

In [2]:
filter_fac_list = [32, 16, 32, 64, 128]  # filters 1/32 degree --> filter_fac/32 degree
cycle_list = ['lorenz', 'bleck', 'bleck', 'bleck', 'bleck']

In [3]:
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar

In [4]:
run = 'nw2_0.03125deg_N15_baseline_hmix20'

In [5]:
basepath = '/glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/'
st = xr.open_dataset('%s/%s/static.nc' % (basepath,run), decode_times=False)

In [7]:
# 500 day averaged energy cycles
workpath = '/glade/work/noraloose/' 

ds_list = []

for filter_fac, cycle in zip(filter_fac_list, cycle_list):
    ds = xr.open_dataset('%s/%s/%s_cycle_fac%i_500days.nc' % (workpath, run, cycle, filter_fac), decode_times=False)
    ds_list.append(ds)

### Integrate over domain

In [18]:
fldlist_nonTWA = [
    'MKE_wind_stress', 'MKE_bottom_drag', 'MKE_vertical_viscosity', 'MKE_horizontal_viscosity',
    'EKE_wind_stress', 'EKE_bottom_drag', 'EKE_vertical_viscosity', 'EKE_horizontal_viscosity',
    'MKE_to_MPE', 'BC_conversion', 'dEPEdt', 'EKE_production',
    'work_eddy_momentum_fluxes'
]
fldlist_TWA = [
    'MKE_wind_stress_TWA', 'MKE_bottom_drag_TWA', 'MKE_vertical_viscosity_TWA', 'MKE_horizontal_viscosity_TWA',
    'EKE_wind_stress_TWA', 'EKE_bottom_drag_TWA', 'EKE_vertical_viscosity_TWA', 'EKE_horizontal_viscosity_TWA',
    'MKE_to_MPE_TWA', 'BC_conversion_TWA', 'dEPEdt', 'EKE_production_TWA',
    'work_eddy_momentum_fluxes_TWA'
]

In [19]:
dst_integrals_list = []
for ds, cycle in zip(ds_list, cycle_list):
    if cycle == 'lorenz':
        fldlist = fldlist_nonTWA
    elif cycle == 'bleck':
        fldlist = fldlist_TWA
    else:
        raise AssertionError('specify valid cycle name')
    dst_integrals = xr.Dataset()
    for name in fldlist:
        dst_integrals[name] = (ds[name] * st.area_t).sum(dim=['xh', 'yh'])
    dst_integrals_list.append(dst_integrals)

In [53]:
for dst_integrals, cycle, filter_fac in zip(dst_integrals_list, cycle_list, filter_fac_list):
    print('{:>7s} {:s} {:s} {:2.1f} {:s}'.format(cycle, 'cycle,', 'filter scale: ', filter_fac / 32, 'degree'))
    print('-----------------------------------------')
    for name in dst_integrals.data_vars:
        print('{:>30s} {:10.2f}'.format(name, dst_integrals[name].values/1e6))  # in GW converted from m5 s-3 by using reference density 1000 kgm-3
    print('')
    print('')

 lorenz cycle, filter scale:  1.0 degree
-----------------------------------------
               MKE_wind_stress     691.60
               MKE_bottom_drag    -232.78
        MKE_vertical_viscosity    -162.72
      MKE_horizontal_viscosity     -12.75
               EKE_wind_stress    -279.38
               EKE_bottom_drag     -71.60
        EKE_vertical_viscosity     107.08
      EKE_horizontal_viscosity     -44.44
                    MKE_to_MPE     422.29
                 BC_conversion     417.74
                        dEPEdt      -1.15
                EKE_production     418.89
     work_eddy_momentum_fluxes    -134.99


  bleck cycle, filter scale:  0.5 degree
-----------------------------------------
           MKE_wind_stress_TWA     403.59
           MKE_bottom_drag_TWA    -262.77
    MKE_vertical_viscosity_TWA     -46.61
  MKE_horizontal_viscosity_TWA      -2.14
           EKE_wind_stress_TWA       8.64
           EKE_bottom_drag_TWA     -41.29
    EKE_vertical_viscosity_TWA    